In [16]:
import pandas as pd
import numpy as np
import random
import os
from os.path import isfile, join
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from modules.many_features.env import SyntheticEnv
from stable_baselines3 import DQN
import warnings
from stable_baselines3.common.vec_env import DummyVecEnv
warnings.filterwarnings('ignore')

In [17]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [18]:
def get_steps(filename, prefix):
    return int(filename[len(prefix):][:-10])

In [19]:
def load_model(filepath, env):
    try:
        return DQN.load(filepath, env=env)
    except Exception as e:
        print(e)

In [20]:
def get_val_metrics(model, validation_env):
    val_df = pd.DataFrame()
    try:
        while True:
            obs, done = validation_env.reset(), False
            while not done:
                action, states = model.predict(obs, deterministic=True)
                obs, rew, done, info = validation_env.step(action)
                if done==True:
                    val_df = val_df.append(info, ignore_index=True)

    except StopIteration:
        pass
    acc, f1, roc_auc, = utils.test(val_df['y_actual'], val_df['y_pred'])
    min_path_length = val_df.episode_length.min()
    average_path_length = val_df.episode_length.mean()
    max_path_length = val_df.episode_length.max()
    min_sample_pathway = val_df[val_df.episode_length==min_path_length].trajectory.iloc[0]
    max_sample_pathway = val_df[val_df.episode_length==max_path_length].trajectory.iloc[0]
    return acc, f1, roc_auc, min_path_length, average_path_length, max_path_length, min_sample_pathway, max_sample_pathway

In [21]:
def create_test_df(model_path, model_type, X_test, y_test):
    testing_env = DummyVecEnv([lambda: SyntheticEnv(X_test, y_test, random=False)])
#     testing_env = LupusEnv(X_test, y_test, random=False)
    model = load_model(model_path, model_type, testing_env)
    test_df = pd.DataFrame()
    try:
        while True:
            obs, done = testing_env.reset(), False
            while not done:
                action, states = model.predict(obs, deterministic=True)
                obs, rew, done, info = testing_env.step(action)
                if done==True:
                    test_df = test_df.append(info, ignore_index=True)

    except StopIteration:
        pass
    min_path_length = test_df.episode_length.min()
    average_path_length = test_df.episode_length.mean()
    max_path_length = test_df.episode_length.max()
    return test_df

In [22]:
def create_val_df(folder, X_val, y_val, prefix):
    best_f1, best_acc, best_roc_auc = -1, -1, -1
    perf_list = []
        
    for item in os.listdir(folder):
        path = join(folder, item)
        #print(path)
        if (isfile(path)) & (path.endswith('.zip')):
#             validation_env = DummyVecEnv([lambda: LupusEnv(X_val, y_val, random=False)])
            validation_env = SyntheticEnv(X_val, y_val, random=False)
            model = load_model(path, validation_env)
            acc, f1, roc_auc, min_path_length, average_path_length, max_path_length, min_sample_pathway, max_sample_pathway = get_val_metrics(model, validation_env)
           
            perf_dict = {'steps': get_steps(item, prefix), 'acc':acc, 'f1':f1, 'roc_auc':roc_auc, 
                          'min_path_length':min_path_length, 'average_path_length':average_path_length, 
                          'max_path_length':max_path_length, 'min_sample_pathway':min_sample_pathway, 
                          'max_sample_pathway':max_sample_pathway} 
            perf_list.append(perf_dict)
            if acc > best_acc:
                best_acc = acc
                model.save(f'{folder}/best_acc_model')
            if f1 > best_f1:
                best_f1 = f1
                model.save(f'{folder}/best_f1_model')
            if roc_auc > best_roc_auc:
                best_roc_auc = roc_auc
                model.save(f'{folder}/best_roc_auc_model')
    
    print('creating dataframe object')
    val_df = pd.DataFrame.from_dict(perf_list) 
    val_df = val_df.sort_values(by=['steps'])
    val_df = val_df.reset_index(drop=True)
    print('saving validation results')
    val_df.to_csv(f'{folder}/validation_results.csv', index=False)
    return val_df

#### Validation

In [23]:
validation_df = pd.read_csv('../../final/data/new_experiments/val_set_constant.csv')
validation_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,9.496631,-1.000000,-1.000000,3.515439,375.685261,101.027779,122.174205,2.820006,1,1.276220,147.925454,74.188240,23.800042,11.545421,97.593498,28.489894,32.520362,1
1,7.092780,336.562609,-1.000000,-1.000000,115.507513,75.569193,35.449822,2.815743,0,0.870413,105.709466,118.836405,30.607322,2.113748,88.833122,21.278341,30.690491,3
2,6.554136,-1.000000,1.774495,6.145810,-1.000000,86.849700,-1.000000,2.263958,0,0.569184,139.078814,-1.000000,-1.000000,-1.000000,-1.000000,19.662409,-1.000000,6
3,12.417159,252.249921,1.701860,0.188413,515.449324,76.102948,-1.000000,4.894880,1,0.576132,4.588230,-1.000000,-1.000000,-1.000000,-1.000000,37.251478,-1.000000,3
4,7.748672,258.549935,2.980693,5.137341,363.214987,82.395181,99.002425,2.821283,1,0.885522,84.419465,33.706997,19.591641,27.675132,69.578682,23.246016,27.257252,5


In [24]:
X_val = validation_df.iloc[:, 0:-1]
y_val = validation_df.iloc[:, -1]
X_val, y_val = np.array(X_val), np.array(y_val)
X_val.shape, y_val.shape

((5600, 17), (5600,))

In [26]:
folder = '../../final/models/new_experiments/logs/trial_3'
prefix = 'dqn_basic_'
val_df = create_val_df(folder, X_val, y_val, prefix)
val_df.head()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

,steps,acc,f1,roc_auc,min_path_length,average_path_length,max_path_length,min_sample_pathway,max_sample_pathway
0,500000,0.096071,0.021913,0.5,3.0,5.167679,10.0,"[hematocrit, ethanol, Inconclusive diagnosis]","[hematocrit, mcv, tibc, hemoglobin, copper, fo..."
1,1000000,0.096071,0.021913,0.5,2.0,2.455714,3.0,"[ethanol, Inconclusive diagnosis]","[ethanol, glucose, Inconclusive diagnosis]"
2,1500000,0.096071,0.021913,0.5,2.0,2.802679,4.0,"[ferritin, Inconclusive diagnosis]","[ferritin, copper, glucose, Inconclusive diagn..."
3,2000000,0.096071,0.021913,0.5,2.0,2.361607,3.0,"[copper, Inconclusive diagnosis]","[copper, tsat, Inconclusive diagnosis]"
4,2500000,0.096071,0.021913,0.5,2.0,2.455714,3.0,"[ethanol, Inconclusive diagnosis]","[ethanol, creatinine, Inconclusive diagnosis]"
